In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install hazm

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hazm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

In [4]:
df=pd.read_csv("/content/drive/MyDrive/Digikala-ProductCategorization/cleaned_dataset.csv")
# df=pd.read_csv("cleaned_dataset.csv")
df.head(3)

,product_id,product_name,has_image,category1,category2,category3
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری


In [5]:
df =df[df['product_name'].notna()]
print(df.isna().sum(), '\n')

product_id      0
product_name    0
has_image       0
category1       0
category2       0
category3       0
dtype: int64 



In [6]:
hazm.word_tokenize(df.iloc[1]["product_name"])

['شورت', 'زنانه', 'کد', 'مجموعه', 'عددی']

In [7]:
df['product_name_cat1_cat2'] = df['category1'].astype(str) +" -"+df['category2'].astype(str) +" -"+ df['product_name'].astype(str)

In [8]:
df= df.rename(columns={"product_id": "product_id", "product_name":"product_item_name" ,"has_image":"has_image","category1":"category1","category2":"category2","category3":"category3","product_name_cat1_cat2":"product_name"})

In [9]:
df['text_len_by_words'] = df['product_name'].apply(lambda t: len(hazm.word_tokenize(t)))
df.head(3)

,product_id,product_item_name,has_image,category1,category2,category3,product_name,text_len_by_words
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه,مد و پوشاک -زنانه و مردانه -جاکارتی طرح هاوایی...,10
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه,مد و پوشاک -زنانه -شورت زنانه کد مجموعه عددی,9
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری,کتاب، لوازم تحریر و هنر -لوازم تحریر -استند لو...,15


In [10]:
def get_name_count(cat_name):
    name, count = np.unique(df[cat_name], return_index=False, return_inverse=False, return_counts=True, axis=None)
    count_sort_ind = np.argsort(count)
    cat3_name=name[count_sort_ind]
    cat3_count=count[count_sort_ind]
    for i in range(len(cat3_name)):
        print(cat3_name[i]+" : "+str(cat3_count[i]))
    return cat3_name,cat3_count

cat3_name,cat3_count=get_name_count("category3")

تخم پرندگان : 1
سینمای خانگی : 1
شاخه گل : 1
یخچال‌های فروشگاهی و فریزرهای بستنی : 1
آویز ساعت : 1
آویز و جاسوییچی : 1
غذاساز و بخارپز کودک و نوزاد : 1
ژاکت و پلیور مردانه : 1
کفش ورزشی بچه گانه : 1
لباس راحتی و خواب بچگانه : 1
ماشین آلات صنعتی : 1
مترجم : 1
پیراهن بچگانه : 1
پوشاک بارداری و شیردهی : 1
پلاتر : 1
مراقبت کودکان : 1
پد امضا : 1
پیش غذا : 1
سویشرت و هودی زنانه : 1
شور : 1
سرویس اینترنت : 1
سالاد تازه : 1
ساعت مچی دیجیتالی : 1
گوشواره : 1
ساعت تندرستی : 1
زیورآلات نقره : 1
دامن زنانه : 1
دکمه سر دست : 1
جت مارت : 1
سمعک و لوازم جانبی : 2
محصولات بدون گلوتن : 2
میوه منجمد : 2
تجهیزات مدرن اداری : 2
تجهیزات پخت نان : 2
نان و رشته محلی : 2
تشک مواج : 2
جعبه جواهرات : 2
قبله نما : 2
حمام آلتراسونیک : 2
سنگ های قیمتی و نیمه قیمتی : 2
فکس : 2
انواع بار و تنقلات مغذی : 2
دکوراسیون و معماری : 2
انگشتر : 2
رهیاب ماهواره‌ای : 2
شیرینی خشک : 2
نوشیدنی غیر لبنی : 2
جوراب بچگانه : 2
ویدئو وال : 2
لوازم تم عکاسی کودک : 2
پیراهن مردانه : 2
کالر آی دی : 2
کاپشن، پالتو و بارانی مردانه : 2
ک

In [11]:
max_idx=0
for i,cnt in enumerate(cat3_count):
    if cnt<=5:
        max_idx=i

In [12]:
for i in range(max_idx+1):
    df = df.drop(df[df['category3'] == cat3_name[i]].index)
print(df.shape)

(529017, 8)


In [13]:
min_max_len = df["text_len_by_words"].min(), df["text_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 5 	Max: 40


In [14]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='text_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [15]:
data_gl_than(df, 256, 5)

Texts with word length of greater than 5 and less than 256 includes 100.00% of the whole!


In [16]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['text_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [17]:
def get_cat_idx(cat,cat_options):
    res=0
    for i,opt in enumerate(cat_options):
        if cat==opt:
            res=i
            break
    return res

In [18]:
category1_options=df["category1"].unique()
category2_options=df["category2"].unique()
category3_options=df["category3"].unique()

In [19]:
df["cat1_id"]=df['category1'].apply(lambda t: get_cat_idx(t,category1_options))
df["cat2_id"]=df['category2'].apply(lambda t: get_cat_idx(t,category2_options))
df["cat3_id"]=df['category3'].apply(lambda t: get_cat_idx(t,category3_options))
df.head(3)

,product_id,product_item_name,has_image,category1,category2,category3,product_name,text_len_by_words,cat1_id,cat2_id,cat3_id
0,dkp-5747199,جاکارتی طرح هاوایی کد,False,مد و پوشاک,زنانه و مردانه,اکسسوری زنانه و مردانه,مد و پوشاک -زنانه و مردانه -جاکارتی طرح هاوایی...,10,0,0,0
1,dkp-2890044,شورت زنانه کد مجموعه عددی,False,مد و پوشاک,زنانه,لباس زنانه,مد و پوشاک -زنانه -شورت زنانه کد مجموعه عددی,9,0,1,1
2,dkp-5999158,استند لوازم اداری رومیزی مدل مجموعه عددی,False,کتاب، لوازم تحریر و هنر,لوازم تحریر,لوازم اداری,کتاب، لوازم تحریر و هنر -لوازم تحریر -استند لو...,15,1,2,2


In [20]:
print(len(category1_options))
print(len(category2_options))
print(len(category3_options))

11
96
575


## Train-Test split

In [21]:
train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['cat3_id'])
train, valid = train_test_split(train, test_size=0.1, random_state=42, stratify=train['cat3_id'])
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['product_name'].values.tolist(), train['cat3_id'].values.tolist()
x_valid, y_valid = valid['product_name'].values.tolist(), valid['cat3_id'].values.tolist()
x_test, y_test = test['product_name'].values.tolist(), test['cat3_id'].values.tolist()


print(train.shape)
print(valid.shape)
print(test.shape)

(428503, 11)
(47612, 11)
(52902, 11)


## Load Tokenizer

In [22]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [24]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16


MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
# OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'
# os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [25]:


label2id = {label: i for i, label in enumerate(category3_options)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'اکسسوری زنانه و مردانه': 0, 'لباس زنانه': 1, 'لوازم اداری': 2, 'سالنامه': 3, 'عرقیات و گلاب': 4, 'هدفون، هدست، میکروفون': 5, 'استیکر و پوستر': 6, 'لوازم جانبی گوشی موبایل': 7, 'روغن و ضد یخ خودرو': 8, 'کتاب هنر و سرگرمی': 9, 'ابزار دستی': 10, 'اکسسوری زنانه': 11, 'ساک و چمدان': 12, 'آرایش چشم': 13, 'کتاب شعر و ادبیات': 14, 'دفتر و کاغذ و مقوا': 15, 'پسرانه': 16, 'سرویس خواب کودک و نوزاد': 17, 'بیسکویت و ویفر': 18, 'پارچ، بطری و لیوان': 19, 'چادر': 20, 'ابزار مهمانی': 21, 'قاب عکس و تابلو': 22, 'ورزش های هوازی و تناسب اندام': 23, 'اکسسوری مردانه': 24, 'قطعات الکترونیکی': 25, 'لباس مردانه': 26, 'دخترانه': 27, 'کفش زنانه': 28, 'مراقبت پوست': 29, 'لوازم تزیینی': 30, 'کفش مردانه': 31, 'لوازم جانبی لپ تاپ': 32, 'ابزارآلات باغبانی': 33, 'زیورآلات طلا زنانه': 34, 'لوازم جانبی ساعت و مچ بند هوشمند': 35, 'دستگیره در': 36, 'اسباب بازی': 37, 'حمام': 38, 'سرویس خواب': 39, 'نور و روشنایی': 40, 'نظافت لباس': 41, 'لوازم جانبی عکاسی و فیلم برداری': 42, 'ملزومات هدیه': 43, 'نوزاد': 44, 'قمقم

In [26]:
len(id2label)

575

In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })



## Dataset

In [31]:
class DigikalaDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Digikala. """

    def __init__(self, tokenizer, product_names, targets=None, max_len=128):
        self.product_names = product_names
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


    def __len__(self):
        return len(self.product_names)

    def __getitem__(self, item):
        product_name = str(self.product_names[item])

        if self.has_target:
            target=self.targets[item]


        encoding = self.tokenizer.encode_plus(
            product_name,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'product_name': product_name,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size):
    dataset = DigikalaDataset(
        product_names=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [32]:
train_data_loader = create_data_loader(train['product_name'].to_numpy(), train['cat3_id'].to_numpy(), tokenizer, max_len=MAX_LEN, batch_size=TRAIN_BATCH_SIZE)
valid_data_loader = create_data_loader(valid['product_name'].to_numpy(), valid['cat3_id'].to_numpy(), tokenizer,max_len= MAX_LEN, batch_size=VALID_BATCH_SIZE)
test_data_loader = create_data_loader(test['product_name'].to_numpy(), test['cat3_id'].to_numpy(), tokenizer, max_len=MAX_LEN, batch_size=TEST_BATCH_SIZE)

In [33]:
len(train_data_loader)

26782

## Model

In [37]:
class Level3Model(nn.Module):

    def __init__(self, config):
        super(Level3Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [38]:
lvl3_model = Level3Model(config=config)
lvl3_model = lvl3_model.to(device)

print('lvl3_model', type(lvl3_model))

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

lvl3_model <class '__main__.Level3Model'>


## Training

In [43]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

In [44]:
def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)
    return y, loss

In [45]:
def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)


            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            _, preds = torch.max(outputs, dim=1)


            loss = loss_fn(outputs, targets)


            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss

In [46]:
def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1
        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']



        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)


        optimizer.zero_grad()


        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)


        _, preds = torch.max(outputs, dim=1)


        loss = loss_fn(outputs, targets)


        losses.append(loss.item())


        loss.backward()

   
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)


        optimizer.step()


        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, optimizer,scheduler,step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [47]:
def eval_callback(epoch, epochs, output_path):
    def eval_cb(model,optimizer,scheduler, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))


            eval_loss_min = eval_loss
            torch.save({'epoch': epoch,'step':step,'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'scheduler_state_dict':scheduler.state_dict(),'eval_loss_min': eval_loss_min,}, output_path+f'level3_checkpoint_ep{epoch}.pt')
   

        return eval_loss_min


    return eval_cb

In [48]:
optimizer = AdamW(lvl3_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



In [49]:
# for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
#     train_y, train_loss, step, eval_loss_min = train_op(
#         model=lvl3_model,
#         data_loader=train_data_loader,
#         loss_fn=loss_fn,
#         optimizer=optimizer,
#         scheduler=scheduler,
#         step=step,
#         print_every_step=EEVERY_EPOCH,
#         eval=True,
#         eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
#         eval_loss_min=eval_loss_min,
#         eval_data_loader=valid_data_loader,
#         clip=CLIP)

#     train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

#     eval_y, eval_loss = eval_op(
#         model=lvl3_model,
#         data_loader=valid_data_loader,
#         loss_fn=loss_fn)

#     eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

#     history['train_acc'].append(train_score['acc'])
#     history['train_loss'].append(train_loss)
#     history['val_acc'].append(eval_score['acc'])
#     history['val_loss'].append(eval_loss)

In [50]:
last_epoch=1
name="level3_checkpoint_ep2_2.pt"
# checkpoint = torch.load(OUTPUT_PATH+f'level2_checkpoint_ep3_{last_epoch}.pt')
checkpoint = torch.load(OUTPUT_PATH+name)
lvl3_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
current_epoch = checkpoint['epoch']
current_epoch=current_epoch+1
step=checkpoint["step"]
eval_loss_min = checkpoint['eval_loss_min']


In [51]:
step

28000

In [52]:
eval_loss_min

0.08072280138692862

In [ ]:
# step=0
# for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
#     train_y, train_loss, step, eval_loss_min = train_op(
#         model=lvl2_model,
#         data_loader=train_data_loader,
#         loss_fn=loss_fn,
#         optimizer=optimizer,
#         scheduler=scheduler,
#         step=step,
#         print_every_step=EEVERY_EPOCH,
#         eval=True,
#         eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
#         eval_loss_min=eval_loss_min,
#         eval_data_loader=valid_data_loader,
#         clip=CLIP)

#     train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

#     eval_y, eval_loss = eval_op(
#         model=lvl2_model,
#         data_loader=valid_data_loader,
#         loss_fn=loss_fn)

#     eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

#     history['train_acc'].append(train_score['acc'])
#     history['train_loss'].append(train_loss)
#     history['val_acc'].append(eval_score['acc'])
#     history['val_loss'].append(eval_loss)

## Testing

In [53]:
def testing(model, data_loader, tokenizer, max_len=128, batch_size=32):

    predictions = []
    true_targets=[]
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)


            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)


            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            true_targets.extend(targets)
            prediction_probs.extend(F.softmax(outputs, dim=1))


    y_true = torch.stack(true_targets).cpu().detach().numpy()
    y_pred = torch.stack(predictions).cpu().detach().numpy()
    y = [y_true, y_pred]

    return y, prediction_probs

In [54]:
test_y, prediction_probs = testing(lvl3_model, test_data_loader, tokenizer, max_len=128)

  0%|          | 0/3307 [00:00<?, ?it/s]

In [57]:
test_score = acc_and_f1(test_y[0], test_y[1], average='weighted')

In [58]:
test_score

{'acc': 0.9792824467884012, 'f1': 0.9775839531167098}

In [ ]:
from sklearn.metrics import classification_report
a=[str(i) for i in range(96)]
print(f'F1: {f1_score(test_y[0], test_y[1], average="weighted")}')
print(classification_report(test_y[0], test_y[1], target_names=a))

## Prediction

In [ ]:
# create a key finder based on label 2 id and id to label
LEVEL1_MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

level1_label2id = {label: i for i, label in enumerate(category1_options)}
level1_id2label = {v: k for k, v in level1_label2id.items()}

print(f'label2id: {level1_label2id}')
print(f'id2label: {level1_id2label}')
# setup the tokenizer and configuration

level1_tokenizer = BertTokenizer.from_pretrained(LEVEL1_MODEL_NAME_OR_PATH)
level1_config = BertConfig.from_pretrained(
    LEVEL1_MODEL_NAME_OR_PATH, **{
        'label2id': level1_label2id,
        'id2label': level1_id2label,
    })



In [60]:
level1_config.num_labels

11

In [61]:
class Level1Model(nn.Module):

    def __init__(self, config):
        super(Level1Model, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(bert_output["pooler_output"])
        logits = self.classifier(pooled_output)
        return logits

In [62]:
lvl1_model = Level1Model(config=level1_config)
lvl1_model = lvl1_model.to(device)

print('lvl1_model', type(lvl1_model))

lvl1_model <class '__main__.Level1Model'>


In [63]:
# level1_last_epoch=1
name="checkpoint_ep2_1.pt"
level1_checkpoint = torch.load(OUTPUT_PATH+name)
lvl1_model.load_state_dict(level1_checkpoint['model_state_dict'])
level1_step=level1_checkpoint["step"]
level1_eval_loss_min = level1_checkpoint['eval_loss_min']

In [ ]:
def predict(model, product_names, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(product_names, None, tokenizer, max_len, batch_size)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']


            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)


            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)


            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
product="تیشرت مردانه"
product2="ساعت مچی عقربه‌ای زنانه "
product3="شامپو بدن مردانه"
product4="قاب گوشی اندروید"
product5="شیر پرچرب"
product6="گوشی موبایل مدل اندروید"
products=[product,product2,product3,product4,product5,product6]
predictions,prediction_probs=predict(lvl1_model,products,tokenizer,max_len=128,batch_size=len(products))

In [ ]:
new_products=[]
for i in range(len(products)):
  category_idx=predictions[i]
  print(products[i])
  print(category1_options[category_idx])
  new_products.append(category1_options[category_idx]+" -"+products[i])
  print("-"*80)

In [ ]:
new_products

In [ ]:
level2_predictions,level2_prediction_probs=predict(lvl3_model,new_products,tokenizer,max_len=128,batch_size=len(new_products))

In [ ]:
for i in range(len(new_products)):
  category2_idx=level2_predictions[i]
  print(new_products[i])
  print(category2_options[category2_idx])
  print("-"*80)